In [1208]:
import pandas as pd
import sqlite3

## connection to the database

In [1209]:
con = sqlite3.connect("../data/checking-logs.sqlite")

## get the schema

In [1210]:
pd.io.sql.read_sql(sql="PRAGMA table_info(test);", con=con)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


 ## get the first 10 rows

In [1211]:
pd.io.sql.read_sql(sql="SELECT * FROM test LIMIT 10", con=con)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## find the minimum value of the delta between the first commit of the user and the deadline

In [1212]:
from time import strftime


df_min = pd.io.sql.read_sql(
    sql="SELECT uid, MIN((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) \
        AS 'MIN(diff)' FROM test \
            JOIN deadlines ON test.labname=deadlines.labs WHERE \
                (labname = 'laba04' OR labname = 'laba04s' OR \
                labname = 'laba05' OR labname = 'laba06' or labname = 'laba06s') \
                    GROUP BY labs ORDER BY 2 ASC LIMIT 1",
    con=con,
)
df_min

,uid,MIN(diff)
0,user_30,-202


## do the same thing, but for the maximum

In [1213]:
df_max = pd.io.sql.read_sql(
    sql="SELECT uid, MAX((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) \
        AS 'MAX(diff)' FROM test \
            JOIN deadlines ON test.labname=deadlines.labs WHERE \
                (labname = 'laba04' OR labname = 'laba04s' OR \
                labname = 'laba05' OR labname = 'laba06' or labname = 'laba06s') \
                    GROUP BY labs ORDER BY 2 DESC LIMIT 1",
    con=con,
)
df_max

,uid,MAX(diff)
0,user_25,-2


## do the same thing but for the average

In [1214]:
df_avg = pd.io.sql.read_sql(
    sql="SELECT AVG(diff) FROM (SELECT uid, AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) \
        AS 'diff' FROM test \
            JOIN deadlines ON test.labname=deadlines.labs WHERE \
                (labname = 'laba04' OR labname = 'laba04s' OR \
                labname = 'laba05' OR labname = 'laba06' or labname = 'laba06s') \
                    GROUP BY labs)",
    con=con,
)
df_avg

,AVG(diff)
0,-91.598932


## test the hypothesis

In [1215]:
views_diff = pd.io.sql.read_sql(
    sql="SELECT test.uid, AVG((strftime('%s', test.first_commit_ts) - deadlines.deadlines) / 3600) \
        AS 'avg_diff', COUNT(pageviews.uid) AS pageviews FROM test \
            JOIN deadlines ON test.labname=deadlines.labs \
            JOIN pageviews ON test.uid=pageviews.uid WHERE \
                (labname = 'laba04' OR labname = 'laba04s' OR \
                labname = 'laba05' OR labname = 'laba06' or labname = 'laba06s')\
                    GROUP BY test.uid",
    con=con,
)
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.185834
pageviews,-0.185834,1.000000


## Close connection

In [1216]:
con.close()